In [1]:
from PIL import Image
import face_recognition
import os
import cv2
import numpy as np

In [3]:
# Creating function to exract frames from a video, in order to train the model
def extract_frames(video_path, dest_path, frame_rate=1):
    """
    Extracts frames from a video.
    
    :param video_path: Path to the video file.
    :param dest_path: Directory where extracted frames will be saved.
    :param frame_rate: Extract every 'frame_rate' frames. 1 means every frame, 2 means every second frame, etc.
    """
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    vidcap = cv2.VideoCapture(video_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    count = 0

    while vidcap.isOpened():
        success, image = vidcap.read()
        if not success:
            break
        if count % frame_rate == 0:
            cv2.imwrite(os.path.join(dest_path, f"frame_{count}.jpg"), image)
        count += 1

    vidcap.release()
    cv2.destroyAllWindows()

In [4]:
# Extracting frames from training video
video_path = '/root/computer_vision/face-recognition/training/willsmith/willsmith1.mp4'
dest_path = '/root/computer_vision/face-recognition/training/willsmith'
extract_frames(video_path, dest_path, frame_rate=5)

In [5]:
image_dir = "/root/computer_vision/face-recognition/training/willsmith"
# Initialize lists to store face encodings and names
known_face_encodings = []
known_face_names = []

# Load training images from the 'training' directory
if os.path.exists(image_dir):
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            print(f"Processing {filename}...")
            image_path = os.path.join(image_dir, filename)
            # Load an image file to find face locations
            image = face_recognition.load_image_file(image_path)
            # Get face encodings for any faces in the uploaded image
            encodings = face_recognition.face_encodings(image)
            if len(encodings) > 0:
                face_encoding = encodings[0]
                known_face_encodings.append(face_encoding)
                known_face_names.append("Will Smith")
            else:
                print(f"No faces found in {filename}")

Processing frame_485.jpg...
Processing frame_455.jpg...
Processing frame_260.jpg...
Processing frame_50.jpg...
Processing frame_325.jpg...
No faces found in frame_325.jpg
Processing frame_385.jpg...
No faces found in frame_385.jpg
Processing frame_375.jpg...
No faces found in frame_375.jpg
Processing frame_10.jpg...
No faces found in frame_10.jpg
Processing frame_415.jpg...
No faces found in frame_415.jpg
Processing frame_420.jpg...
No faces found in frame_420.jpg
Processing frame_480.jpg...
Processing frame_470.jpg...
No faces found in frame_470.jpg
Processing frame_150.jpg...
No faces found in frame_150.jpg
Processing frame_305.jpg...
No faces found in frame_305.jpg
Processing frame_70.jpg...
No faces found in frame_70.jpg
Processing frame_350.jpg...
No faces found in frame_350.jpg
Processing frame_335.jpg...
No faces found in frame_335.jpg
Processing frame_65.jpg...
Processing frame_330.jpg...
No faces found in frame_330.jpg
Processing frame_120.jpg...
Processing frame_280.jpg...
Pr

In [6]:
print("Initializing...")

video_path = '/root/computer_vision/face-recognition/testing/willsmith/willsmithtest1.mp4'
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('face_rec_output.mp4',
                      fourcc, 30.0, (frame_width, frame_height))

print(f"Loaded {len(known_face_encodings)} face encodings.")

# Initialize lists to store face locations, encodings, and names
face_locations = []
face_encodings = []
face_names = []
# Initialize variable to control frame processing
process_this_frame = True

print("Starting video processing...")

# Loop over frames from the video file stream
while cap.isOpened():
    # Read the next frame from the file
    ret, frame = cap.read()
    # If the frame was not grabbed, then we have reached the end of the stream
    if not ret:
        print("Reached end of video.")
        break

    # Only process every other frame of video to save time
    if process_this_frame:
        print("Processing frame...")
        # Resize frame of video for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        small_frame = frame 

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(small_frame)
        print(f"Found {len(face_locations)} face(s) in this frame.")

        face_encodings = face_recognition.face_encodings(
            small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(
                known_face_encodings, face_encoding, tolerance=0.6)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(
                known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)
        print(f"Identified faces: {face_names}")

    # Switch to not process the next frame
    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35),
                      (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6),
                    font, 1.0, (255, 255, 255), 1)

    # Write the resulting image to the output video file
    out.write(frame)

print("Releasing video objects...")
# Release the file pointers
out.release()
cap.release()

Initializing...
Loaded 50 face encodings.
Starting video processing...
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['Will Smith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['Will Smith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['Will Smith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['Will Smith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['Will Smith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['Will Smith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['Will Smith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['Will Smith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['Will Smith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['Will Smith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['Will Smith']
Proce